In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

### Device configuration

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### Hyper-parameters 

In [3]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 128
learning_rate = 0.001

### MNIST dataset 

In [4]:
train_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=True, 
    transform=transforms.ToTensor(),
    download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=False, 
    transform=transforms.ToTensor()
)

### Data loader

In [5]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, 
    batch_size=batch_size, 
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, 
    batch_size=batch_size, 
    shuffle=False
)

### Fully connected neural network with one hidden layer

In [6]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

### Loss and optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Train the model

In [8]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/469], Loss: 0.3463
Epoch [1/5], Step [200/469], Loss: 0.3836
Epoch [1/5], Step [300/469], Loss: 0.2738
Epoch [1/5], Step [400/469], Loss: 0.1502
Epoch [2/5], Step [100/469], Loss: 0.0911
Epoch [2/5], Step [200/469], Loss: 0.1166
Epoch [2/5], Step [300/469], Loss: 0.0916
Epoch [2/5], Step [400/469], Loss: 0.1190
Epoch [3/5], Step [100/469], Loss: 0.1100
Epoch [3/5], Step [200/469], Loss: 0.0262
Epoch [3/5], Step [300/469], Loss: 0.0659
Epoch [3/5], Step [400/469], Loss: 0.0917
Epoch [4/5], Step [100/469], Loss: 0.0165
Epoch [4/5], Step [200/469], Loss: 0.1072
Epoch [4/5], Step [300/469], Loss: 0.0227
Epoch [4/5], Step [400/469], Loss: 0.1026
Epoch [5/5], Step [100/469], Loss: 0.0316
Epoch [5/5], Step [200/469], Loss: 0.0146
Epoch [5/5], Step [300/469], Loss: 0.1035
Epoch [5/5], Step [400/469], Loss: 0.0284


### Test the model

In [9]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.0 %
